Sumário: 

1. Physics-Informed Neural Networks (PINNs) com PyTorch

1. **Introdução às PINNs**
    - O que são PINNs?
    - Por que combinar física com redes neurais?
    - Aplicações em finanças e outras áreas.

2. **Fundamentos Matemáticos das PINNs**
    - Formulação de equações diferenciais parciais (PDEs) em PINNs.
    - Função de perda combinando dados e física.
    - Exemplo simples: Equação do Calor ou Black-Scholes.

3. **Implementação Básica com PyTorch**
    - Arquitetura da rede neural.
    - Definição da função de perda (dados + PDE).
    - Treinamento e desafios numéricos.

4. **Aplicação em Finanças: Modelo Black-Scholes**
    - Revisão da equação Black-Scholes.
    - Adaptação para uma PINN.
    - Comparação com soluções analíticas/númericas tradicionais.

5. **Tópicos Avançados e Otimizações**
    - Tratamento de condições iniciais/de contorno.
    - Balanceamento de termos na função de perda.
    - Aceleração com GPUs e técnicas de treinamento.

6. **Estudo de Caso Prático**
    - Implementação completa de uma PINN para opções europeias.
    - Visualização de resultados.

7. **Limitações e Extensões**
    - Quando PINNs falham?
    - Alternativas híbridas (ex: PINNs + SDEs).

## 1. Physics-Informed Neural Networks (PINNs) com PyTorch


### 1.1 **Introdução às PINNs**

1.1 **Introdução às PINNs**
- O que são PINNs?
- Por que combinar física com redes neurais?
- Aplicações em finanças e outras áreas.

O que são PINNs?
PINNs são redes neurais treinadas para resolver equações diferenciais (PDEs/ODEs) incorporando diretamente as leis físicas (ou financeiras) na função de perda. Em vez de depender apenas de dados, elas usam a estrutura conhecida da equação subjacente (ex: Black-Scholes) como regularização.

Por que combinar física com redes neurais?
- Dados escassos: Em problemas financeiros ou físicos, dados reais podem ser limitados ou ruidosos. A física atua como um "guia" para generalização.
- Interpretabilidade: A solução respeita leis conhecidas, mesmo em regiões sem dados.
- Flexibilidade: PINNs lidam com problemas não-lineares e geometrias complexas sem malhas numéricas.

Aplicações em finanças:
- Precificação de opções (Black-Scholes).
- Calibração de modelos estocásticos.
- Simulação de cenários de mercado com restrições teóricas.


### 2. **Fundamentos Matemáticos das PINNs**


**Fundamentos Matemáticos das PINNs**
    - Formulação de equações diferenciais parciais (PDEs) em PINNs.
    - Função de perda combinando dados e física.
    - Exemplo simples: Equação do Calor ou Black-Scholes.

As PINNs resolvem problemas baseados em equações diferenciais (PDEs/ODEs) combinando duas fontes de informação:  
1. **Dados observados** (ex: preços de opções no mercado).  
2. **Leis físicas/financeiras** (ex: equação Black-Scholes).  


#### **2.1 Formulação Geral de uma PINN**  
Considere uma PDE genérica:  
$$
\mathcal{F}[u(t, x)] = 0, \quad t \in [0, T], x \in \Omega  
$$  
com:  
- **Condição inicial (IC):** \( u(0, x) = u_0(x) \).  
- **Condições de contorno (BC):** \( u(t, x) = g(t, x) \) em \( \partial\Omega \).  

**Exemplo (Equação Black-Scholes):**  
$$
\frac{\partial V}{\partial t} + rS\frac{\partial V}{\partial S} + \frac{1}{2}\sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} - rV = 0  
$$  
Onde \( V(t, S) \) é o preço da opção, \( S \) é o ativo subjacente, \( r \) é a taxa livre de risco, e \( \sigma \) é a volatilidade.  

A ordem lógica do cógido se da pela seguinte forma:

Código Completo da PINN para Black-Scholes
1. Importações e Configurações Iniciais
2. Definição da Arquitetura da PINN
3. Funções de Amostragem de Dados
4. Funções de Perda
5. Loop de Treinamento


Ordem Lógica de Execução
- Configuração: Define parâmetros do modelo e hiperparâmetros da rede.
- 
- Arquitetura: Cria a PINN com camadas lineares e ativação Tanh.
- 
- Amostragem: Gera pontos para o domínio, condição inicial e contorno.
- 
- Perdas: Calcula os termos da PDE, condição inicial e contorno.
- 
- Treinamento: Minimiza a perda total via backpropagation.


# A Equação de Black-Scholes

A equação de Black-Scholes é uma PDE (Equação Diferencial Parcial) que descreve a evolução do preço de uma opção europeia sob certas premissas, como:
- Mercado eficiente;
- Sem pagamento de dividendos;
- Volatilidade constante.

Para uma **opção de compra (call)**, sua fórmula analítica é:

$$
C(S_t, t) = S_t N(d_1) - K e^{-r(T-t)} N(d_2)
$$

Onde:
- \( C(S_t, t) \) é o preço da opção no tempo \( t \), dado o preço do ativo \( S_t \).
- \( N(\cdot) \) é a função cumulativa da distribuição normal padrão.
- \( d_1 \) e \( d_2 \) são termos que dependem de \( S_t \), \( K \), \( r \), \( \sigma \) e \( T - t \).

### Forma Diferencial (PDE) da Equação de Black-Scholes

A forma diferencial da equação, que é a base para a PINN, é:

$$
\frac{\partial V}{\partial t} + r S \frac{\partial V}{\partial S} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} - rV = 0
$$

Essa equação é a base para diversas abordagens numéricas e de aprendizado de máquina, como redes neurais físicas (*Physics-Informed Neural Networks* - PINNs), que buscam resolver a PDE de forma eficiente.

In [ ]:
#Código para Treinamento de uma Rede Neural Profunda (PINN) para o Modelo Black-Scholes
#1. Importações e Configurações Iniciais
import torch
import torch.nn as nn
import numpy as np

# Parâmetros do modelo Black-Scholes
r = 0.05      # Taxa livre de risco
sigma = 0.2   # Volatilidade
K = 100.0     # Preço de exercício (strike)
T = 1.0       # Tempo até a expiração (em anos)
S_min = 0     # Preço mínimo do ativo
S_max = 200   # Preço máximo do ativo

# Hiperparâmetros da PINN
layers = [2, 50, 50, 1]  # Arquitetura da rede: [input_dim, hidden_dim, ..., output_dim]
lambda_pde = 1.0         # Peso da perda da PDE
lambda_ic = 1.0          # Peso da condição inicial
lambda_bc = 1.0          # Peso das condições de contorno
batch_size = 100         # Número de pontos por batch
epochs = 5000            # Número de épocas

#2. Definição da Arquitetura da PINN
class PINN(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.linear_layers = nn.ModuleList(
            [nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        )
        self.activation = nn.Tanh()  # Ativação suave para gradientes estáveis

    def forward(self, t, S):
        inputs = torch.cat([t, S], dim=1)  # Concatena tempo (t) e preço (S)
        for layer in self.linear_layers[:-1]:
            inputs = self.activation(layer(inputs))
        output = self.linear_layers[-1](inputs)  # Sem ativação na última camada
        return output
    
#3. Funções de Amostragem de Dados
def sample_domain(batch_size):
    """Gera pontos aleatórios no domínio [0,T] x [S_min, S_max]."""
    t = torch.rand(batch_size, 1) * T
    S = torch.rand(batch_size, 1) * (S_max - S_min) + S_min
    return t, S

def sample_ic(batch_size):
    """Gera pontos para a condição inicial (payoff em t=T)."""
    S_ic = torch.rand(batch_size, 1) * (S_max - S_min) + S_min
    return S_ic

def sample_bc_times(batch_size):
    """Gera tempos para as condições de contorno."""
    return torch.rand(batch_size, 1) * T

#4. Funções de Perda
def loss_pde(t, S, V_hat):
    """Calcula o residual da PDE Black-Scholes."""
    t.requires_grad_(True)
    S.requires_grad_(True)
    
    # Derivadas parciais via autograd
    dV_dt = torch.autograd.grad(V_hat.sum(), t, create_graph=True)[0]
    dV_dS = torch.autograd.grad(V_hat.sum(), S, create_graph=True)[0]
    d2V_dS2 = torch.autograd.grad(dV_dS.sum(), S, create_graph=True)[0]
    
    # Residual da PDE
    residual = dV_dt + r * S * dV_dS + 0.5 * (sigma ** 2) * (S ** 2) * d2V_dS2 - r * V_hat
    return (residual ** 2).mean()

def loss_ic(S_ic):
    """Perda da condição inicial (payoff final)."""
    t_ic = torch.ones_like(S_ic) * T  # Tempo final (t=T)
    V_true_ic = torch.max(S_ic - K, torch.zeros_like(S_ic))  # Payoff: max(S-K, 0)
    V_pred_ic = pinn(t_ic, S_ic)
    return ((V_pred_ic - V_true_ic) ** 2).mean()

def loss_bc(t_bc):
    """Perda das condições de contorno."""
    # BC1: V(t, S=0) = 0
    S_bc_low = torch.zeros_like(t_bc)
    V_bc_low = pinn(t_bc, S_bc_low)
    loss_bc1 = (V_bc_low ** 2).mean()
    
    # BC2: V(t, S -> ∞) ≈ S - K*e^{-r(T-t)}
    S_bc_high = torch.ones_like(t_bc) * S_max
    V_bc_high = pinn(t_bc, S_bc_high)
    V_true_bc_high = S_bc_high - K * torch.exp(-r * (T - t_bc))
    loss_bc2 = ((V_bc_high - V_true_bc_high) ** 2).mean()
    
    return loss_bc1 + loss_bc2

#5. Loop de Treinamento
# Inicializa a rede e o otimizador
pinn = PINN(layers)
optimizer = torch.optim.Adam(pinn.parameters(), lr=0.001)

for epoch in range(epochs):
    optimizer.zero_grad()
    
    # Amostra pontos do domínio e calcula as perdas
    t_pde, S_pde = sample_domain(batch_size)
    V_hat = pinn(t_pde, S_pde)
    
    l_pde = loss_pde(t_pde, S_pde, V_hat)
    l_ic = loss_ic(sample_ic(batch_size))
    l_bc = loss_bc(sample_bc_times(batch_size))
    
    # Perda total ponderada
    loss = lambda_pde * l_pde + lambda_ic * l_ic + lambda_bc * l_bc
    
    # Backpropagation
    loss.backward()
    optimizer.step()
    
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}, PDE Loss: {l_pde.item():.4f}")


In [14]:
%pip install matplotlib

   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------- ----------------------------- 2.1/8.1 MB 11.7 MB/s eta 0:00:01
   ---------------------- ----------------- 4.5/8.1 MB 11.7 MB/s eta 0:00:01
   ----------------------------------- ---- 7.1/8.1 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 11.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------------------------------- ---- 2.4/2.7 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 11.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#Código para Treinamento de uma Rede Neural Profunda (PINN) para o Modelo Black-Scholes

# ======================
# 1. Configuração
# ======================
r = 0.05       # Taxa livre de risco
sigma = 0.2    # Volatilidade
K = 100.0      # Preço de exercício
T = 1.0        # Tempo até a maturidade (anos)
S_min = 0      # Preço mínimo do ativo
S_max = 200    # Preço máximo do ativo

# Hiperparâmetros da PINN
layers = [2, 50, 50, 1]  # Estrutura da rede [entrada, ocultas..., saída]
lambda_pde = 1.0         # Peso da perda da PDE
lambda_ic = 1.0          # Peso da perda da condição inicial
lambda_bc = 1.0          # Peso da perda da condição de contorno
batch_size = 100         # Pontos por lote
epochs = 5000            # Número de épocas de treino
learning_rate = 0.001    # Taxa de aprendizado do otimizador

# ======================
# 2. Arquitetura da PINN
# ======================
class BlackScholesPINN(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.linear_layers = nn.ModuleList(
            [nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        )
        self.activation = nn.Tanh()  # Função de ativação suave para estabilidade

    def forward(self, t, S):
        t_norm = t.view(-1, 1) / T
        S_norm = S.view(-1, 1) / K
        inputs = torch.cat([t_norm, S_norm], dim=1)

        for layer in self.linear_layers[:-1]:
            inputs = self.activation(layer(inputs))
        return self.linear_layers[-1](inputs)

# ======================
# 3. Amostragem de Dados
# ======================
def sample_domain(batch_size):
    t = torch.rand(batch_size, 1) * T
    S = torch.rand(batch_size, 1) * (S_max - S_min) + S_min
    t.requires_grad = True
    S.requires_grad = True
    return t, S

def sample_ic(batch_size):
    S_ic = torch.rand(batch_size, 1) * (S_max - S_min) + S_min
    return S_ic

def sample_bc(batch_size):
    t_bc = torch.rand(batch_size, 1) * T
    return t_bc

# ======================
# 4. Funções de Perda
# ======================
def compute_derivatives(t, S, V_hat):
    dV_dt = torch.autograd.grad(V_hat.sum(), t, create_graph=True)[0]
    dV_dS = torch.autograd.grad(V_hat.sum(), S, create_graph=True)[0]
    d2V_dS2 = torch.autograd.grad(dV_dS.sum(), S, create_graph=True)[0]
    return dV_dt, dV_dS, d2V_dS2

def pde_loss(t, S, V_hat):
    dV_dt, dV_dS, d2V_dS2 = compute_derivatives(t, S, V_hat)
    residual = dV_dt + r*S*dV_dS + 0.5*(sigma**2)*(S**2)*d2V_dS2 - r*V_hat
    return (residual**2).mean()

def ic_loss(pinn, S_ic):
    t_ic = torch.ones_like(S_ic) * T
    V_true = torch.max(S_ic - K, torch.zeros_like(S_ic))
    V_pred = pinn(t_ic, S_ic)
    return ((V_pred - V_true)**2).mean()

def bc_loss(pinn, t_bc):
    S_bc1 = torch.zeros_like(t_bc)
    V_bc1 = pinn(t_bc, S_bc1)
    loss1 = (V_bc1**2).mean()

    S_bc2 = torch.ones_like(t_bc) * S_max
    V_true_bc2 = S_bc2 - K * torch.exp(-r * (T - t_bc))
    V_bc2 = pinn(t_bc, S_bc2)
    loss2 = ((V_bc2 - V_true_bc2)**2).mean()

    return loss1 + loss2

# ======================
# 5. Loop de Treinamento
# ======================
pinn = BlackScholesPINN(layers)
optimizer = torch.optim.Adam(pinn.parameters(), lr=learning_rate)

loss_history = []

for epoch in range(epochs):
    optimizer.zero_grad()

    t_pde, S_pde = sample_domain(batch_size)
    V_hat = pinn(t_pde, S_pde)

    l_pde = pde_loss(t_pde, S_pde, V_hat)
    l_ic = ic_loss(pinn, sample_ic(batch_size))
    l_bc = bc_loss(pinn, sample_bc(batch_size))

    loss = lambda_pde*l_pde + lambda_ic*l_ic + lambda_bc*l_bc
    loss_history.append(loss.item())

    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f"Época {epoch:5d} | Perda Total: {loss.item():.4e} | "
              f"Perda PDE: {l_pde.item():.4e} | Perda IC: {l_ic.item():.4e} | "
              f"Perda BC: {l_bc.item():.4e}")

Época     0 | Perda Total: 1.2887e+04 | Perda PDE: 1.0179e-02 | Perda IC: 2.3319e+03 | Perda BC: 1.0555e+04
Época  1000 | Perda Total: 3.0909e+03 | Perda PDE: 9.0591e+00 | Perda IC: 2.8809e+02 | Perda BC: 2.7937e+03
Época  2000 | Perda Total: 5.7378e+02 | Perda PDE: 1.9597e+01 | Perda IC: 2.8109e+01 | Perda BC: 5.2608e+02
Época  3000 | Perda Total: 9.5734e+01 | Perda PDE: 8.6751e+00 | Perda IC: 8.4406e+00 | Perda BC: 7.8618e+01
Época  4000 | Perda Total: 1.6335e+01 | Perda PDE: 9.0750e-01 | Perda IC: 1.0315e+00 | Perda BC: 1.4396e+01


In [ ]:
#(Com func. de aviso) Código para Treinamento de uma Rede Neural Profunda (PINN) para o Modelo Black-Scholes

def log_step(step):
    print(f"✅ Etapa concluída: {step}")

try:
    # ======================
    # 1. Configuração
    # ======================

    # a) Parâmetros do Modelo Black-Scholes
    r = 0.05       # Taxa livre de risco
    sigma = 0.2    # Volatilidade
    K = 100.0      # Preço de exercício
    T = 1.0        # Tempo até a maturidade (anos)
    S_min = 0      # Preço mínimo do ativo
    S_max = 200    # Preço máximo do ativo

    # b)Hiperparâmetros da PINN
    layers = [2, 50, 50, 1]
    lambda_pde = 1.0
    lambda_ic = 1.0
    lambda_bc = 1.0
    batch_size = 100
    epochs = 5000
    learning_rate = 0.001

    log_step("Configuração inicial")

    # ======================
    # 2. Arquitetura da PINN
    # ======================
    class BlackScholesPINN(nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.linear_layers = nn.ModuleList(
                [nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]
            )
            self.activation = nn.Tanh()

        def forward(self, t, S):
            t_norm = t.view(-1, 1) / T
            S_norm = S.view(-1, 1) / K
            inputs = torch.cat([t_norm, S_norm], dim=1)
            
            for layer in self.linear_layers[:-1]:
                inputs = self.activation(layer(inputs))
            return self.linear_layers[-1](inputs)

    log_step("Definição da arquitetura da rede")

    # ======================
    # 3. Amostragem de Dados
    # ======================
    def sample_domain(batch_size):
        t = torch.rand(batch_size, 1) * T
        S = torch.rand(batch_size, 1) * (S_max - S_min) + S_min
        t.requires_grad = True
        S.requires_grad = True
        return t, S

    def sample_ic(batch_size):
        S_ic = torch.rand(batch_size, 1) * (S_max - S_min) + S_min
        return S_ic

    def sample_bc(batch_size):
        t_bc = torch.rand(batch_size, 1) * T
        return t_bc

    log_step("Amostragem de dados")

    # ======================
    # 4. Funções de Perda
    # ======================
    def compute_derivatives(t, S, V_hat):
        dV_dt = torch.autograd.grad(V_hat.sum(), t, create_graph=True)[0]
        dV_dS = torch.autograd.grad(V_hat.sum(), S, create_graph=True)[0]
        d2V_dS2 = torch.autograd.grad(dV_dS.sum(), S, create_graph=True)[0]
        return dV_dt, dV_dS, d2V_dS2

    def pde_loss(t, S, V_hat):
        dV_dt, dV_dS, d2V_dS2 = compute_derivatives(t, S, V_hat)
        residual = dV_dt + r*S*dV_dS + 0.5*(sigma**2)*(S**2)*d2V_dS2 - r*V_hat
        return (residual**2).mean()

    def ic_loss(pinn, S_ic):
        t_ic = torch.ones_like(S_ic) * T
        V_true = torch.max(S_ic - K, torch.zeros_like(S_ic))
        V_pred = pinn(t_ic, S_ic)
        return ((V_pred - V_true)**2).mean()

    def bc_loss(pinn, t_bc):
        S_bc1 = torch.zeros_like(t_bc)
        V_bc1 = pinn(t_bc, S_bc1)
        loss1 = (V_bc1**2).mean()

        S_bc2 = torch.ones_like(t_bc) * S_max
        V_true_bc2 = S_bc2 - K * torch.exp(-r * (T - t_bc))
        V_bc2 = pinn(t_bc, S_bc2)
        loss2 = ((V_bc2 - V_true_bc2)**2).mean()

        return loss1 + loss2

    log_step("Definição das funções de perda")

    # ======================
    # 5. Loop de Treinamento
    # ======================
    pinn = BlackScholesPINN(layers)
    optimizer = torch.optim.Adam(pinn.parameters(), lr=learning_rate)

    loss_history = []

    log_step("Inicialização da rede e otimizador")

    for epoch in range(epochs):
        try:
            optimizer.zero_grad()

            t_pde, S_pde = sample_domain(batch_size)
            V_hat = pinn(t_pde, S_pde)

            l_pde = pde_loss(t_pde, S_pde, V_hat)
            l_ic = ic_loss(pinn, sample_ic(batch_size))
            l_bc = bc_loss(pinn, sample_bc(batch_size))

            loss = lambda_pde*l_pde + lambda_ic*l_ic + lambda_bc*l_bc
            loss_history.append(loss.item())

            loss.backward()
            optimizer.step()

            if epoch % 1000 == 0:
                print(f"Época {epoch:5d} | Perda Total: {loss.item():.4e} | "
                      f"Perda PDE: {l_pde.item():.4e} | Perda IC: {l_ic.item():.4e} | "
                      f"Perda BC: {l_bc.item():.4e}")
            
        except Exception as e:
            print(f"❌ Erro na época {epoch}: {e}")
            break  # Encerra o loop caso ocorra um erro

    log_step("Treinamento concluído")

except Exception as e:
    print(f"❌ Ocorreu um erro durante a execução: {e}")

✅ Etapa concluída: Configuração inicial
✅ Etapa concluída: Definição da arquitetura da rede
✅ Etapa concluída: Amostragem de dados
✅ Etapa concluída: Definição das funções de perda
✅ Etapa concluída: Inicialização da rede e otimizador
Época     0 | Perda Total: 1.2130e+04 | Perda PDE: 3.0908e-02 | Perda IC: 1.5536e+03 | Perda BC: 1.0576e+04
Época  1000 | Perda Total: 3.0298e+03 | Perda PDE: 6.2679e+00 | Perda IC: 2.3360e+02 | Perda BC: 2.7899e+03
Época  2000 | Perda Total: 5.8488e+02 | Perda PDE: 1.7007e+01 | Perda IC: 3.0456e+01 | Perda BC: 5.3742e+02
Época  3000 | Perda Total: 1.0111e+02 | Perda PDE: 2.4611e+01 | Perda IC: 1.3841e+01 | Perda BC: 6.2657e+01
Época  4000 | Perda Total: 1.4024e+01 | Perda PDE: 2.4893e+00 | Perda IC: 2.6192e+00 | Perda BC: 8.9155e+00
✅ Etapa concluída: Treinamento concluído


### . Detalhes do Código

#### 1. Configurações Iniciais

1.a. Parâmetros do Modelo Black-Scholes:

````
r = 0.05       # Taxa livre de risco
sigma = 0.2    # Volatilidade
K = 100.0      # Preço de exercício (strike)
T = 1.0        # Tempo até a maturidade (em anos)
S_min = 0      # Preço mínimo do ativo
S_max = 200    # Preço máximo do ativo
````
**Váriaveis representadas no código:**

- **r = Taxa Livre de Risco:**
    Representa a taxa livre de risco, ou Taxa de retorno sem risco, (r na PDE). Usada para descontar o valor futuro da opção( usada para descontar fluxos futuros ). Use a taxa anualizada de títulos públicos de prazo semelhante ao vencimento da opção, por exemplo, se a opção vence em 1 ano, pegue a taxa SELIC ou treasury bond de 1 ano. As melhores faixas são o r variando entre 1% a 15% a.a., dependendo do país e cenário econômico. Para Ajuste, pode ser atualizada periodicamente para refletir a taxa de mercado mais recente.Pode ser fixada ou modelada estocasticamente em casos mais avançados.
    Modelagem Estocástica da Taxa Livre de Risco

     Como seria uma modelagem estocástica da taxa definida?
     A **taxa livre de risco** \( r \) normalmente é considerada constante para simplificar modelos de precificação. No entanto, em mercados mais complexos ou de longo prazo, essa taxa pode ser modelada como um **processo    estocástico**.
    Modelo de Vasicek:
        Um modelo amplamente utilizado para taxas de juros é o **Modelo de Vasicek**, que segue a equação diferencial   estocástica:

    $$
    d r_t = a (b - r_t) dt + \sigma_r dW_t
    $$

    Onde:
    - \( a \) → **Velocidade de reversão à média**: determina o quão rápido \( r_t \) converge para a taxa de longo     prazo.
    - \( b \) → **Taxa de longo prazo (média)**: nível médio ao qual a taxa tende a se estabilizar.
    - \( sigma_r \) → **Volatilidade da taxa**: grau de incerteza na variação da taxa de juros.
    - \( dW_t \) → **Movimento Browniano**: captura as variações aleatórias ao longo do tempo.

    Impacto da Modelagem Estocástica
    Ao invés de considerar \( r \) como constante, simulamos **trajetórias possíveis** para a taxa de juros ao longo    do tempo. Isso permite:
    - Capturar cenários onde a taxa **sobe** ou **abaixa** dinamicamente.
    - Melhorar a modelagem de **opções de longo prazo** e **títulos de renda fixa**, tornando as simulações mais    realistas.

---
- sigma = Volatilidade do Ativo Subjacente:
    Volatilidade do ativo subjacente (σ na PDE). Controla a difusão do preço, ou seja, mede o desvio padrão das variações do ativo. Uma boa prática é usar a volatilidade histórica ou a volatilidade implícita (se disponível no mercado de opções). Faixa comum é para Ações estáveis: 10% a 30% a.a. e Ações voláteis ou criptomoedas: 50% a 150% a.a. Para um ajuste, podemos calcular com séries históricas de preços:
````
volatilidade = np.std(retornos) * np.sqrt(252)
````
Ou use a volatilidade implícita das opções negociadas no mercado (melhor prática para precificação realista).

Como seria usar a volatilidade implícita das opções negociadas no mercado?
A volatilidade implícita é aquela que, se colocada no modelo (por exemplo, no Black-Scholes), faz com que o preço teórico da opção bata com o preço negociado no mercado. Para obter, devemos coletar os preços de opções no mercado para diferentes strikes e vencimentos. Além disso devemos usar um método de busca numérica (ex: Newton-Raphson) para encontrar a σ que iguala o preço do modelo ao preço de mercado.
Exemplo de conceito (simplificado):

````
def erro_volatilidade(sigma):
    preco_modelo = black_scholes(S, K, T, r, sigma)
    return preco_modelo - preco_mercado

# encontrar a sigma que zera o erro
sigma_implicita = optimize.brentq(erro_volatilidade, 0.01, 2)
````
Esse tipo de operação tem como impacto refletir as expectativas de mercado sobre a volatilidade futura e uma forma mais precisa para precificar opções reais do que a volatilidade histórica.

---

- K = Preço de Exercício (Strike)
    Preço de exercício (strike) da opção que representa o valor de compra/venda acordado na opção. Aparece na condição inicial (payoff final). Depende do produto financeiro. Pode ser ATM (At-the-money), ITM (In-the-money) ou OTM (Out-the-money).
    - O que é a condição inicial (payoff final):
        Na equação de derivadas parciais (PDE) da precificação de opções, a condição inicial é o valor da opção no momento do vencimento (t = T), ou seja, o payoff.
        - Função de Payoff para Opções
            Opção de Compra (*Call*) - A função de payoff para uma opção de compra (*call*) é definida como:
            $$
            V(S, T) = \max(S_T - K, 0)
            $$
            Onde:
                - \( S_T \) → Preço do ativo no tempo de vencimento \( T \).
                - \( K \) → Preço de exercício da opção.
                - A função **máximo** garante que o payoff seja **zero** caso \( S_T < K \), pois o titular da opção não a exerceria.
            
            Opção de Venda (*Put*) - A função de payoff para uma opção de venda (*put*) é definida como:

            $$
            V(S, T) = \max(K - S_T, 0)
            $$

            Onde:
                - \( S_T \) → Preço do ativo no tempo de vencimento \( T \).
                - \( K \) → Preço de exercício da opção.
                - A função **máximo** garante que o payoff seja **zero** caso \( S_T > K \), pois o titular da opção não a          exerceria.

    Este modelo representa a base para a precificação de opções e pode ser utilizado em diversos métodos,incluindo simulações estocásticas e equações diferenciais parciais.

- O que é ATM, ITM e OTM?
    - ATM (At-the-money):
     - Quando o preço do ativo ≈ strike.
    - ITM (In-the-money):
        - Para call: preço do ativo > strike
        - Para put: preço do ativo < strike
    - OTM (Out-the-money):
        - Para call: preço do ativo < strike
        - Para put: preço do ativo > strike

    Em simulações, use múltiplos valores de K para construir a curva de valor da opção. Para isso, você pode precificar a opção para uma série de valores de strike e montar uma curva V(K) com o seguinte exemplo:
    ````
    Ks = np.linspace(80, 120, 20)
    valores_opcao = [black_scholes(S, K, T, r, sigma) for K in Ks]

    plt.plot(Ks, valores_opcao)
    plt.xlabel("Preço de Exercício (K)")
    plt.ylabel("Valor da Opção")
    plt.title("Curva Valor da Opção x Strike")
    plt.show()
    ````
    As vantagens de se utilizar isso é que permite uma analise de opções sob diferentes nivéis de strike usando para contruir superfícies de volatilidade e estudar smiles ou skews
    A faixa comum se da nas casas próximas ao preço atual do ativo, ou a ±10%, ±20%, etc., para cenários de stress. Defina de acordo com o contrato ou com a faixa que deseja testar.

---

- T = Tempo até o Vencimento
    Tempo total até a expiração da opção. Define o domínio temporal t∈[0,T]. É conveniente que se expresse em anos (ex: 6 meses = 0.5). Prefira ajustes dinâmicos se for atualizar o valor da opção ao longo do tempo. Comum operar na faixa de 1 dia (≈ 0.00396 anos) até 5 anos (5.0). O ajuste é feito pelo contrato da opção e em modelos dinâmicos, diminua t continuamente conforme o tempo passa, uma vez que modelos com atualização dinâmica são mais realistas para opções com vencimento curto e o valor da opção decai com o tempo (efeito theta)

    Ex: hoje T = 1 ano

    Amanhã T = 1 - (1/252)

    Em simulação contínua:

    𝑡 → 𝑇 − passo de tempo acumulado

---

- S_min e S_max = — Limites do Domínio Espacial
    Intervalo de preços considerado na simulação. Importante para as condições de contorno. S_min: geralmente 0 e S_max: no mínimo 2x o preço atual, mas prefira 3x ou 4x para evitar problemas de contorno. Exemplo:
    - Se preço atual = 100:
        - S_min = 0
        - S_max = 200 a 400

---


1.b. Hiperparâmetros da PINN

#### 2. Definição da Arquitetura da PINN

#### 3. Funções de Amostragem de Dados

#### 4. Funções de Perda

#### 5. Loop de Treinamento

#### Próximos Passos

- Visualização: Plotar a solução da PINN vs. solução analítica de Black-Scholes.
- Extensões: Adicionar dados de mercado reais (loss_data) para calibração.

In [ ]:
# Exemplo de visualização (após o treinamento)
import matplotlib.pyplot as plt

S_test = torch.linspace(S_min, S_max, 100).reshape(-1, 1)
t_test = torch.zeros_like(S_test)  # t=0 (hoje)
V_pred = pinn(t_test, S_test).detach().numpy()

plt.plot(S_test, V_pred, label="PINN Solution")
plt.xlabel("Preço do Ativo (S)"); plt.ylabel("Preço da Opção (V)")
plt.legend(); plt.show()

In [ ]:
# ======================
# 6. Visualization
# ======================
# Plot training loss
plt.figure(figsize=(10, 5))
plt.semilogy(loss_history)
plt.xlabel("Epoch")
plt.ylabel("Loss (log scale)")
plt.title("Training Convergence")
plt.grid(True)
plt.show()

# Plot option price surface
with torch.no_grad():
    
    # Preço da opção (V) em função do preço do ativo (S) e do tempo (t)
    S_test = torch.linspace(S_min, S_max, 100).reshape(-1, 1)
    t_test = torch.zeros_like(S_test)  # t=0 (hoje)
    V_pred = pinn(t_test, S_test).detach().numpy()
    
    plt.plot(S_test, V_pred, label="PINN Solution")
    plt.xlabel("Preço do Ativo (S)"); plt.ylabel("Preço da Opção (V)")
    plt.legend(); plt.show()    

    #Training Convergence
    S_test = torch.linspace(S_min, S_max, 100).reshape(-1, 1)
    t_test = torch.linspace(0, T, 50).reshape(-1, 1)
    

    
    # Create grid for 3D plot
    S_grid, t_grid = torch.meshgrid(S_test.squeeze(), t_test.squeeze())
    V_grid = pinn(t_grid.reshape(-1, 1), S_grid.reshape(-1, 1))
    V_grid = V_grid.reshape(S_grid.shape).numpy()
    
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(S_grid.numpy(), t_grid.numpy(), V_grid, cmap='viridis')
    ax.set_xlabel('Asset Price (S)')
    ax.set_ylabel('Time to Maturity (t)')
    ax.set_zlabel('Option Price (V)')
    ax.set_title('PINN Solution to Black-Scholes')
    plt.show()
